<a href="https://colab.research.google.com/github/rajeevfromkrec/deep_learning/blob/master/consumer_complaint_classification_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mv complaints.csv.zip  complaints.zip


In [70]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.0MB/s 


In [19]:
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from chart_studio.plotly import iplot
from plotly.offline import iplot
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip

--2020-06-03 13:04:41--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 52.84.52.62, 52.84.52.49, 52.84.52.72, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|52.84.52.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259123057 (247M) [binary/octet-stream]
Saving to: ‘complaints.csv.zip’

complaints.csv.zip  100%[===================>] 247.12M  55.4MB/s    in 5.0s    

2020-06-03 13:04:46 (49.8 MB/s) - ‘complaints.csv.zip’ saved [259123057/259123057]



In [0]:
zfile=zipfile.ZipFile('complaints.csv.zip')
zfile.extractall()

In [5]:
!ls

complaints.csv	complaints.csv.zip  sample_data


In [1]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"rajeevranjankrec","key":"5e8e081c0d2d711641f31c7629e1f703"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [3]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets list

- path is now set to: {/content}
ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge             COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-06-03 00:51:53          71785  
roche-data-science-coalition/uncover                          UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53          10483  
kushshah95/the-insurance-company-tic-benchmark                The Insurance Company (TIC) Benchmark               262KB  2020-05-27 16:36:41              1  
kianwee/agricultural-raw-material-prices-19902020             Agricultural Raw Material prices (1990-2020)         23KB  2020-05-27 04:51:29              0  
jessicali9530/anima

In [29]:
!kaggle datasets download -d cfpb/us-consumer-finance-complaints -p /content/

 94% 79.0M/84.5M [00:01<00:00, 53.5MB/s]
100% 84.5M/84.5M [00:01<00:00, 53.1MB/s]


In [0]:
df=pd.read_csv('complaints.csv')

In [7]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2020-05-21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",TX,774XX,NaN,NaN,Web,2020-05-21,In progress,Yes,NaN,3662670
1,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
2,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
3,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
4,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198


In [9]:
df.describe(include='all')

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
count,1620048,1620048,1384883,1620047,1049753,539626,627028,1620048,1592571,1465551,219552,980630,1620048,1620048,1620047,1620048,768477,1.620048e+06
unique,3106,18,76,166,218,504341,10,5638,63,24059,3,4,6,3055,8,2,2,NaN
top,2017-09-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,300XX,Servicemember,Consent provided,Web,2017-09-08,Closed with explanation,Yes,No,NaN
freq,3553,410717,404129,261667,146255,1035,470933,170186,218937,10571,112470,539968,1233538,3387,1267286,1583246,620099,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.247232e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.059373e+06
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.404372e+06
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.562108e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.139190e+06


In [10]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    410717
Mortgage                                                                        302693
Debt collection                                                                 295736
Credit reporting                                                                140432
Credit card                                                                      89190
Bank account or service                                                          86206
Credit card or prepaid card                                                      78055
Checking or savings account                                                      64366
Student loan                                                                     58146
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            17452
Money transfer, virtual currency, or money 

In [11]:
df=df[pd.notnull(df['Consumer complaint narrative'])]
df['Product'].value_counts()


Credit reporting, credit repair services, or other personal consumer reports    170710
Debt collection                                                                 114423
Mortgage                                                                         65151
Credit card or prepaid card                                                      36814
Credit reporting                                                                 31588
Student loan                                                                     26150
Checking or savings account                                                      21510
Credit card                                                                      18838
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             9143
Money transfer, virtual currency, or money 

In [12]:
df['Consumer complaint narrative']

1          transworld systems inc. \nis trying to collect...
3          I would like to request the suppression of the...
4          Over the past 2 weeks, I have been receiving e...
12         I was sold access to an event digitally, of wh...
13         While checking my credit report I noticed thre...
                                 ...                        
1620042    On XXXX XXXX, 2015, I contacted XXXX XXXX, who...
1620043    I can not get from chase who services my mortg...
1620044    I made a payment to CITI XXXX Credit Card on X...
1620045    cfbp i would Like to file a complaint on Exper...
1620046    My husband and I are in the middle of an FHA S...
Name: Consumer complaint narrative, Length: 539626, dtype: object

In [0]:
df.loc[df['Product']== 'Credit reporting','Product']= 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product']== 'Credit card','Product']='Credit card or prepaid card'
df.loc[df['Product']== 'Prepaid card','Product']='Credit card or prepaid card'
df.loc[df['Product']== 'Payday loan','Product']='Payday loan, title loan, or personal loan'
df.loc[df['Product']== 'Virtual currency','Product']='Money transfer, virtual currency, or money service'
df=df[df.Product !='Other financial service' ]

In [14]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    202298
Debt collection                                                                 114423
Mortgage                                                                         65151
Credit card or prepaid card                                                      57102
Student loan                                                                     26150
Checking or savings account                                                      21510
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             9143
Payday loan, title loan, or personal loan                                         8945
Money transfer, virtual currency, or money service                                8757
Money transfers                            

In [16]:
df['Consumer complaint narrative']

1          transworld systems inc. \nis trying to collect...
3          I would like to request the suppression of the...
4          Over the past 2 weeks, I have been receiving e...
12         I was sold access to an event digitally, of wh...
13         While checking my credit report I noticed thre...
                                 ...                        
1620042    On XXXX XXXX, 2015, I contacted XXXX XXXX, who...
1620043    I can not get from chase who services my mortg...
1620044    I made a payment to CITI XXXX Credit Card on X...
1620045    cfbp i would Like to file a complaint on Exper...
1620046    My husband and I are in the middle of an FHA S...
Name: Consumer complaint narrative, Length: 539334, dtype: object

In [0]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)

In [0]:
vocab_size=50000
seq_length=100
embedded_dim=100

tokenizer= Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_{|}~',lower=True)



In [0]:
tokenizer.fit_on_texts(df['Consumer complaint narrative'].values)

In [0]:
word_index=tokenizer.word_index

In [24]:
len(word_index)

190994

In [25]:
word_index

{'credit': 1,
 'account': 2,
 'report': 3,
 'information': 4,
 'would': 5,
 'payment': 6,
 'loan': 7,
 'debt': 8,
 'bank': 9,
 'told': 10,
 'received': 11,
 'company': 12,
 'card': 13,
 'time': 14,
 'called': 15,
 'reporting': 16,
 'never': 17,
 'payments': 18,
 'sent': 19,
 'letter': 20,
 'also': 21,
 'back': 22,
 'pay': 23,
 'get': 24,
 'paid': 25,
 'call': 26,
 'mortgage': 27,
 'said': 28,
 'made': 29,
 'accounts': 30,
 'amount': 31,
 'one': 32,
 'due': 33,
 'number': 34,
 'could': 35,
 'phone': 36,
 'days': 37,
 'money': 38,
 'balance': 39,
 'consumer': 40,
 'collection': 41,
 'late': 42,
 'still': 43,
 'asked': 44,
 'since': 45,
 'date': 46,
 'nt': 47,
 'even': 48,
 'please': 49,
 'dispute': 50,
 'years': 51,
 'name': 52,
 'file': 53,
 'make': 54,
 'request': 55,
 'home': 56,
 'contacted': 57,
 'us': 58,
 'check': 59,
 'interest': 60,
 'month': 61,
 'service': 62,
 'removed': 63,
 'months': 64,
 'new': 65,
 'agency': 66,
 'help': 67,
 'complaint': 68,
 'times': 69,
 'address': 70,

In [26]:
print( 'found %s unique tokens.' %len(word_index))

found 190994 unique tokens.


In [0]:
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=seq_length)

In [28]:
X.shape

(539334, 100)

In [29]:
Y = pd.get_dummies(df['Product']).values
Y.shape

(539334, 12)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(485400, 100) (485400, 12)
(53934, 100) (53934, 12)


In [31]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedded_dim,input_length=X.shape[1]),
                                    tf.keras.layers.SpatialDropout1D(0.2),
                                    tf.keras.layers.LSTM(100,dropout=0.2,recurrent_dropout=0.2,activation='relu'),
                                    tf.keras.layers.Dense(12,activation='softmax')


])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 12)                1212      
Total params: 5,081,612
Trainable params: 5,081,612
Non-trainable params: 0
_________________________________________________________________


In [0]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
 726/6826 [==>...........................] - ETA: 32:35 - loss: 2.6477 - accuracy: 0.3785

In [0]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [0]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [0]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show();

In [0]:
new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
print(pred, labels[np.argmax(pred)])